# Build A Spam Filter With Naive Bayes

In this project the challenge was to apply Bayes theorem to create an algorithm with >80% accuracy to sort out spam messages.

The dataset used was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

For convention spam messages are classified as "spam" whereas the counterpart is be called "ham".

As a conclusion, the developed algorithm achieved a 98% accuracy rate.

Finally, this exercise was developed by the team at [Dataquest](https://www.dataquest.io/). 

# 1. Importing and inspecting the data

In [1]:
import pandas as pd

data = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label','SMS'])

data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
data.shape

(5572, 2)

In [3]:
data['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Our dataset is approximately 87% Ham and 13% Spam

# 2. Creating Training and Testing sets

The 5,572 messages will be divided as follows:

1) Training Set (80% = 4,458 messages)

2) Testing Set (20% = 1,114 messages)

The next step is to randomize the entire dataset to ensure that spam and ham messages are spread properly.

In [4]:
randomized_data = data.sample(frac=1, random_state=1)

split_index = round(len(randomized_data) * 0.8)

training_data = randomized_data[:split_index].reset_index(drop=True)
test_data = randomized_data[split_index:].reset_index(drop=True)


print(training_data.shape)
print(test_data.shape)

(4458, 2)
(1114, 2)


Now we analyze both datasets to verify the distribution of spam and ham messages

In [5]:
print(training_data['Label'].value_counts(normalize=True))

print(test_data['Label'].value_counts(normalize=True))

ham     0.86541
spam    0.13459
Name: Label, dtype: float64
ham     0.868043
spam    0.131957
Name: Label, dtype: float64


As verified, both datasets have similar distributions, so we are good to continue the project.

# Data Cleaning and Preparation

Now it's time to lowercase all messages as well as removing ponctuations and other characters that have no statistical significance for our algorithm.

In [6]:
training_data.head() # Before our cleaning

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [7]:
training_data['SMS'] = training_data['SMS'].str.replace('\W',' ')

training_data['SMS'] = training_data['SMS'].str.lower()

training_data.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [8]:
vocabulary=[]

training_data['SMS'] = training_data['SMS'].str.split()

for x in training_data['SMS']:
    for y in x:
        vocabulary.append(y)
        
vocabulary=list(set(vocabulary))

len(vocabulary)

7783

In summary, our test data contains 7783 unique words.

Next it's time to create a dictionary counting the number of times each word appears.

In [9]:
word_counts_per_sms = {unique_word: [0] * len(training_data['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_data['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
word_counts = pd.DataFrame(word_counts_per_sms)

word_counts.head()

,gentleman,lambda,temple,proper,3230,soul,pages,ummifying,failed,abel,...,gotten,step,census,feathery,groovy,anthony,bay,thankyou,hee,psp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
training_set_clean = pd.concat([training_data, word_counts], axis=1)

training_set_clean.head()

,Label,SMS,gentleman,lambda,temple,proper,3230,soul,pages,ummifying,...,gotten,step,census,feathery,groovy,anthony,bay,thankyou,hee,psp
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Creating the spam filter

In order to use Naive Bayes, we need to calculate some probabilities.

In [11]:
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

alpha = 1

p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

n_vocabulary = len(vocabulary)

Now it's time to calculate the probability of each word in our vocabulary given the message is Spam or Ham

In [12]:
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_words:0 for unique_words in vocabulary}

for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

# Classifying a new message

We have all parameters necessary for our algorithm to work.

The filter can be understood as a function that:

Takes in as input a new message (w1, w2, ..., wn).

Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).

Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:

* If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.

* If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.

* If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.


In [13]:
def classify(message):
    message = message.replace('\W',' ')
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
   
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [14]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

'spam'

In [15]:
classify("Sounds good, Tom, then see u there")

'ham'

Time to test the algorithm in the whole test set with 1,114 rows.

In [16]:
test_data.head()

,Label,SMS
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."


In [17]:
test_data['predicted'] = test_data['SMS'].apply(classify)

test_data.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we measure the accuracy of our filter.

In [18]:
len(test_data)

1114

In [19]:
test_data['Label']

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
1109     ham
1110     ham
1111     ham
1112    spam
1113     ham
Name: Label, Length: 1114, dtype: object

In [20]:
test_data.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [21]:
for row in test_data:
    print(row)

Label
SMS
predicted


In [22]:
correct = 0

total = len(test_data)

for row in test_data.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
accuracy = (correct / total)*100

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy: {}%'.format(round((correct/total)*100)))

Correct: 1095
Incorrect: 19
Accuracy: 98%


Our results were even better than expected. Instead of 80% accuracy, we got 98%.